# 2.2

In [1]:
## 2.2
test_string = "hello! こんにちは!"
utf8_encoded = test_string.encode("utf-8")
print(utf8_encoded)
# b'hello! \xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf!'

print(type(utf8_encoded))
# <class 'bytes'>

# Get the byte values for the encoded string (integers from 0 to 255).
print(list(utf8_encoded))
# [104, 101, 108, 108, 111, 33, 32, 227, 129, 147, 227, 130, 147, 227, 129, 171, 227, 129,
# 161, 227, 129, 175, 33]

# One byte does not necessarily correspond to one Unicode character!
print(len(test_string))
# 13

print(len(utf8_encoded))
# 23

print(utf8_encoded.decode("utf-8"))

b'hello! \xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf!'
<class 'bytes'>
[104, 101, 108, 108, 111, 33, 32, 227, 129, 147, 227, 130, 147, 227, 129, 171, 227, 129, 161, 227, 129, 175, 33]
13
23
hello! こんにちは!


In [4]:
# Problem a
# testing utf-16, utf-32
# same string gets longer -> sequeunce inefficiency

utf8_encoded = test_string.encode("utf-8")
print(len(utf8_encoded), utf8_encoded)

utf16_encoded = test_string.encode("utf-16")
print(len(utf16_encoded), utf16_encoded)

utf32_encoded = test_string.encode("utf-32")
print(len(utf32_encoded), utf32_encoded)

23 b'hello! \xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf!'
28 b'\xff\xfeh\x00e\x00l\x00l\x00o\x00!\x00 \x00S0\x930k0a0o0!\x00'
56 b'\xff\xfe\x00\x00h\x00\x00\x00e\x00\x00\x00l\x00\x00\x00l\x00\x00\x00o\x00\x00\x00!\x00\x00\x00 \x00\x00\x00S0\x00\x00\x930\x00\x00k0\x00\x00a0\x00\x00o0\x00\x00!\x00\x00\x00'


In [8]:
# Problem b
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):
    return "".join([bytes([b]).decode("utf-8") for b in bytestring])

# wrong case
try:
    decode_utf8_bytes_to_str_wrong("hello 👋".encode("utf-8"))
except Exception as e:
    print(str(e))

'utf-8' codec can't decode byte 0xf0 in position 0: unexpected end of data


In [20]:
hex(4)
# b'\xc2'

'0x4'

### Problem C
Due to utf-8 encoding rules:
```
UTF-8 uses 1 to 4 bytes to encode Unicode characters. Each byte in a valid sequence must follow specific bit patterns:
	•	1-byte (ASCII): 0xxxxxxx
	•	2-byte: 110xxxxx 10xxxxxx
	•	3-byte: 1110xxxx 10xxxxxx 10xxxxxx
	•	4-byte: 11110xxx 10xxxxxx 10xxxxxx 10xxxxxx

In multi-byte sequences, the first byte determines how many bytes follow, and continuation bytes must begin with 10.
```

`xff` (11111111) is invalid

In [23]:
# Problem c - 2 byte sequence that doesn't decode to any Unicode

encoded = b'\xff\xff'
try:
    print(encoded.decode('utf-8'))
except Exception as e:
    print(type(e), str(e))

<class 'UnicodeDecodeError'> 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte


# 2.5 training bpe

## optimizing the merging step
```
only pair counts that change after each merge are those that overlap with the merged pair
index the counts of all pars -> incrementally update the counts

```

Reference: Linked-list based method
* https://github.com/huggingface/tokenizers/issues/1400


In [35]:
''.join(('a', 'b'))

'ab'

In [43]:
## test small scale
from collections import Counter

tokens = ['a', 'a', 'a', 'b', 'd', 'a', 'a', 'a', 'b', 'a', 'c']

# try merging 2 times
pair_counts = Counter()
pair_positions = {} # start positions
merges = {}

for i in range(1, len(tokens)):
    a = tokens[i-1]
    b = tokens[i]
    
    if (a,b) not in pair_positions:
        pair_positions[(a,b)]=[i-1]
    else:
        pair_positions[(a,b)].append(i-1)
        
    pair_counts[(a,b)]+=1
    
# pair_counts.most_common()
target_pair = max(pair_counts, key=pair_counts.get)
print(target_pair)
new_vocab_token = ''.join(target_pair)

del pair_counts[target_pair]

## Merge
new_tokens = []
prev_idx = 0
for pos in pair_positions[target_pair]:
    print(pos)
    if pos>0:
        # Decrement
        x = (tokens[pos-1], target_pair[0])
        pair_positions[x].remove(pos-1)
        if x!=target_pair:
            pair_counts[x]-=1
        
        # Increment
        x = (tokens[pos-1], new_vocab_token)
        pair_counts[x]+=1
        
    if pos+2<len(tokens):
        # Decrement
        x = (target_pair[1], tokens[pos+2])
        pair_positions[x].remove(pos+1)
        if x!=target_pair:
            pair_counts[x]-=1
        
        # Increment 
        x = (new_vocab_token, tokens[pos+2])
        pair_counts[x]+=1
        
    new_tokens.extend(tokens[prev_idx:pos])
    new_tokens.append(new_vocab_token)
    prev_idx = pos+2
    
new_tokens.extend(tokens[prev_idx:])
print(new_tokens)
pair_counts

('a', 'a')
0
5
['aa', 'a', 'b', 'd', 'aa', 'a', 'b', 'a', 'c']


Counter({('a', 'b'): 2,
         ('aa', 'a'): 2,
         ('b', 'd'): 1,
         ('b', 'a'): 1,
         ('a', 'c'): 1,
         ('d', 'aa'): 1,
         ('d', 'a'): 0})